# First exploration of selected files

In [46]:
import pandas as pd
import numpy as np
import re

In [ ]:
# import dtale

We obtain the csv file with our categorization of the files in our dataset.

In [27]:
file_categorization = pd.read_csv("file-categorization.csv", sep = ';')

In [28]:
by_disorder_countries = {}
not_disorder_specific_countries = {}
survey_countries = {}

for i in range (len(file_categorization)):
    row = file_categorization.iloc[i]
    filename, category = row['filename'], row['category']
    if not (category == "unused" or category == "unsure"):
        if category == "by-disorder":
            df = pd.read_csv("mental-health-dataset/" + filename)
            by_disorder_countries[filename] = sorted(list(set(list(df['Entity']))))
        elif category == "not-disorder-specific":
            not_disorder_specific_countries[filename] = sorted(list(set(list(df['Entity']))))
        elif category == "survey":
            survey_countries[filename] = sorted(list(set(list(df['Entity']))))

In [29]:
all_entities = []

### Checking for cohesive country list in the category `by-disorder`

In [30]:
for file in by_disorder_countries:
    country_list = by_disorder_countries[file]
    all_entities = all_entities + country_list
    print(country_list)
    

['Afghanistan', 'African Region (WHO)', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo', 'Cook Islands', 'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Democratic Republic of Congo', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'East Asia & Pacific (WB)', 'East Timor', 'Eastern Mediterranean Region (WHO)', 'Ecuador', 'Egypt', 'El Salvador', 'England', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia', 'Europe & Central Asia (WB)', 'European Region (WHO)', 'Fiji', 'Finland', 'France', 'G20', 'G

Our lists of countries do not match, meaning that we do not have all of the information for all of the countries (we'll have gaps/holes).

### Checking for cohesive country list in the category `not-disorder-specific`

In [31]:
for file in not_disorder_specific_countries:
    country_list = not_disorder_specific_countries[file]
    all_entities = all_entities + country_list
    print(country_list)

['Abkhazia', 'Afghanistan', 'Africa', 'Africa (IHME GBD)', 'Africa (UN)', 'Akrotiri and Dhekelia', 'Aland Islands', 'Albania', 'Algeria', 'America (IHME GBD)', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antarctica', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Asia', 'Asia (IHME GBD)', 'Asia (UN)', 'Australia', 'Austria', 'Austria-Hungary', 'Azerbaijan', 'Baden', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Bavaria', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'Bonaire Sint Eustatius and Saba', 'Bosnia and Herzegovina', 'Botswana', 'Bouvet Island', 'Brazil', 'British Indian Ocean Territory', 'British Virgin Islands', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Channel Islands', 'Chile', 'China', 'Christmas Island', 'Cocos Islands', 'Colombia', 'Comoros', 'Congo', 'Cook Islands', 'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba'

Same lists!

### Checking for cohesive country list in the category `survey`

In [32]:
for file in survey_countries:
    country_list = survey_countries[file]
    all_entities = all_entities + country_list
    print(country_list)

['Abkhazia', 'Afghanistan', 'Africa', 'Africa (IHME GBD)', 'Africa (UN)', 'Akrotiri and Dhekelia', 'Aland Islands', 'Albania', 'Algeria', 'America (IHME GBD)', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antarctica', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Asia', 'Asia (IHME GBD)', 'Asia (UN)', 'Australia', 'Austria', 'Austria-Hungary', 'Azerbaijan', 'Baden', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Bavaria', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'Bonaire Sint Eustatius and Saba', 'Bosnia and Herzegovina', 'Botswana', 'Bouvet Island', 'Brazil', 'British Indian Ocean Territory', 'British Virgin Islands', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Channel Islands', 'Chile', 'China', 'Christmas Island', 'Cocos Islands', 'Colombia', 'Comoros', 'Congo', 'Cook Islands', 'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba'

Same lists!

## Dividing our different entities

We can see from our lists, that we not only have information about each country. There's records for entire continents, for continents but from different sources... Let's try and assess exactly what entites we have:

First, let's build a dataframe with all of the entities we have.

In [36]:
all_entities = sorted(list(set(all_entities)))
entities = pd.DataFrame({'Entity': all_entities})

,Entity
0,Abkhazia
1,Afghanistan
2,Africa
3,Africa (IHME GBD)
4,Africa (UN)
...,...
329,Yemen People's Republic
330,Yugoslavia
331,Zambia
332,Zanzibar


Now let's create new columns with the additional information we have seen our entities contain:

In [48]:
pattern = r"\((.*?)\)"
entities['source'] = entities['Entity'].str.extract(pattern)
# dtale.show(entities)

In [52]:
entities[~entities['source'].isnull()]

,Entity,source
3,Africa (IHME GBD),IHME GBD
4,Africa (UN),UN
5,African Region (WHO),WHO
10,America (IHME GBD),IHME GBD
21,Asia (IHME GBD),IHME GBD
22,Asia (UN),UN
81,East Asia & Pacific (WB),WB
84,Eastern Mediterranean Region (WHO),WHO
94,Ethiopia (former),former
96,Europe & Central Asia (WB),WB


In [54]:
dtale.show(entities[entities['source'].isnull()])